In [76]:
using DataFrames;
using JuMP;
using Chemostat;
using Chemostat.Tools;
using Plots;
plotly();

## Loading Model

In [77]:
S, mets, rxns = Chemostat.v1.Models.EColi.load_ecoli_v1();
println("S $(size(S)), mets $(size(mets)) rxns $(size(rxns))");

S (72, 95), mets (72, 2) rxns (95, 6)


In [78]:
find((row) -> contains(uppercase(row[:id]), "ATPM"), eachrow(rxns))

0-element Array{Int64,1}

In [79]:
rxns[11,:]

,id,lb,ub,ap,an,t
1,ATPS4r,-1000,1000,0.012151,0.012151,0


In [80]:
mets[S[:,11].nzind,:]

,id,c
1,adp,0.0
2,atp,0.0
3,h2o,0.0
4,h,0.0
5,h[e],100000.0
6,pi,0.0


In [81]:
ATPMmets = S[:,11]

72-element SparseVector{Float64,Int64} with 6 stored entries:
  [13]  =  -1.0
  [17]  =  1.0
  [41]  =  1.0
  [43]  =  3.0
  [44]  =  -4.0
  [60]  =  -1.0

In [82]:
ATPMmets.nzind

6-element Array{Int64,1}:
 13
 17
 41
 43
 44
 60

## Test at ξmax

In [85]:
ξmax = 3e6;#3e6
md = Chemostat.v1.Models.EColi.DEFAULT_MAINTINANCE_DEMAND_FLUX_VALUE; #8e-5;
cost = Chemostat.v1.Models.EColi.DEFAULT_ϕ;#1e-4
fbares = Chemostat.v1.FBA.fba_chemostat_v1(S, mets, rxns, ξmax; ϕub = cost, man_demand_flux_value = md);
println();
println("Results")
println("ϕ: $(getvalue(fbares.ϕ))");
println("Obj: $(getvalue(fbares.obj))");
println();
print("Non zero transport fluxes: ")
println(filter((x) -> x[2] != 0 ,map((x,y) -> (x,y), rxns[:id][find((row) -> row[:t] > 0, eachrow(rxns))],
    Chemostat.v1.FBA.flux.(fbares, rxns[:id][find((row) -> row[:t] > 0, eachrow(rxns))]))))

Academic license - for non-commercial use only

Results
ϕ: 4.307531874058204e-6
Obj: -3.814643210348387e-8

Non zero transport fluxes: Tuple{String,Float64}[("CO2t", -1.73441e-5), ("GLCpts", 8.3e-6), ("H2Ot", -2.2932e-5), ("O2t", 1.64832e-5)]


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] uuid4(::MersenneTwister) at ./deprecated.jl:57
 [3] msg_header at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:18 [inlined]
 [4] msg_pub(::IJulia.Msg, ::String, ::Dict{String,String}, ::Dict{String,Any}) at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:30 (repeats 2 times)
 [5] send_stream(::String) at /Users/Pereiro/.julia/v0.6/IJulia/src/stdio.jl:172
 [6] send_stdio(::String) at /Users/Pereiro/.julia/v0.6/IJulia/src/stdio.jl:130
 [7] (::Base.##302#303{IJulia.#send_stdout,Timer})() at ./event.jl:436
while loading In[85], in expression starting on line 11


In [86]:
ξrange = Tools.logrange(0.001,50, ξmax);
println("ξrange length: $(length(ξrange)), start $(start(ξrange)), last $(last(ξrange))")

ξrange length: 255, start 1, last 3.0e6


## Solving

In [87]:
fbaress = [Chemostat.v1.FBA.fba_chemostat_v1(S,mets,rxns,ξ; man_demand_flux_value = md, ϕub = cost) for ξ in ξrange];

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic lice

## Substrate and Waste

In [88]:
Sid = "glc-D[e]";
Sindx = 35;
Stid = "GLCpts";
Stinx = 49;#50
Wid = "ac[e]";
Windx = 6;
Wtid = "ACt2r";
Wtindx = 6;

## S vs ξ

In [89]:
spdata = Chemostat.v1.FBA.s.(fbaress,Sid,Stid);
sp = plot((ξrange, spdata), xaxis = :log, lw = 2)

## W vs ξ

In [90]:
wpdata = Chemostat.v1.FBA.s.(fbaress,Wid,Wtid);
wp = plot((ξrange,wpdata), xaxis = :log, lw = 2)

## μ vs ξ

In [91]:
τ = 0.000003;
mupdata = Chemostat.v1.FBA.μ.(fbaress, wpdata, τ);
mup = plot((ξrange, mupdata), xaxis = :log, lw =2)

## X vs ξ

In [92]:
Xpdata = Chemostat.v1.FBA.X.(fbaress,mupdata,1)
Xp = plot((ξrange, Xpdata), xaxis = :log, lw = 2)

## X vs D (τ > 0)

In [93]:
X_vs_Dpdata = plot((mupdata, Xpdata),lw = 2)

## X vs D (τ = 0)

In [94]:
τ = 0.0;
mupdatant = Chemostat.v1.FBA.μ.(fbaress, wpdata, τ);
Xpdatant = Chemostat.v1.FBA.X.(fbaress,mupdatant,1);
X_vs_Dpdata = plot((mupdatant, Xpdatant), lw = 2)